<a href="https://colab.research.google.com/github/audreyemmely/pln/blob/main/pln_lista3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Resolução Lista 3**

---

Audrey Emmely Rodrigues Vasconcelos

Karen Nayara Gomes da Silva

In [1]:
#from google.colab import files
#uploaded = files.upload()
#upando train_data e test_data

###### Questão 1
Escolha 10 documentos da base, e determine seu documento mais parecido
(que não pode ser ele mesmo), usando:

a) Representação vetorial CountVectorizer com similaridade do cosseno;

b) Representação vetorial TF-IDF com similaridade do cosseno.

Mostre pelo menos um trecho de cada par de documentos parecidos que justifique
essa semelhança.

In [2]:
import pandas as pd
import numpy as np
import random
import re
import nltk
from IPython.core.display import display, HTML
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [3]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
df_train = pd.read_csv('train_data.txt', sep =':::', header = None, engine = 'python', nrows = 10000)
df_train.columns = ['id', 'title', 'genre', 'description']
df_train['label'] = 'train'
df_test = pd.read_csv('test_data.txt', sep =':::', header = None, engine = 'python', nrows = 10000)
df_test.columns = ['id', 'title', 'description']
df_test['label'] = 'test'

In [5]:
df_train.drop(['id', 'genre'], axis=1, inplace=True)

In [6]:
df_test.drop('id', axis=1, inplace=True)

In [7]:
dataset = pd.concat([df_train, df_test], ignore_index=True)

In [8]:
def preprocess(description):
  description = re.sub(r'\w*\d\w*', '', description) #remove todas as palavras que contêm números
  description = re.sub(r'[^a-zA-Z ]', '', description.lower())
  return re.sub(r'\s+', ' ', description) #retira espaços repetidos

In [9]:
dataset.description = dataset.description.apply(preprocess)

In [10]:
stop_words = set(nltk.corpus.stopwords.words('english'))
def remove_stopwords(description):
    tokenized_text = nltk.word_tokenize(description, language='english')
    return " ".join([token for token in tokenized_text if token not in stop_words])

In [11]:
dataset['description'] = dataset.description.apply(remove_stopwords)

In [12]:
dataset

,title,description,label
0,Oscar et la dame rose (2009),listening conversation doctor parents yearold ...,train
1,Cupid (1997),brother sister past incestuous relationship cu...,train
2,"Young, Wild and Wonderful (1980)",bus empties students field trip museum natural...,train
3,The Secret Sin (1915),help unemployed father make ends meet edith tw...,train
4,The Unrecovered (2007),films title refers unrecovered bodies ground z...,train
...,...,...,...
19995,9/11: Millionaire Widows (2008),wake events nearly families slated receive nea...,test
19996,Gangsters of the Frontier (1944),tex put kern gang away returned reinforcements...,test
19997,Saheb Bahadur (1977),small town situated picturesque indian valley ...,test
19998,Sitting in Limbo (1997),easter sunday sunday school classroom argument...,test


In [13]:
tfidf_X = TfidfVectorizer().fit_transform(dataset.description)
count_X = CountVectorizer().fit_transform(dataset.description)

tfidf_similarity_matrix = cosine_similarity(tfidf_X)
count_similarity_matrix = cosine_similarity(count_X)

In [28]:
np.fill_diagonal(tfidf_similarity_matrix, 0.0)
np.fill_diagonal(count_similarity_matrix, 0.0)

result = [
    {
        'doc': dataset.iloc[i],
        'tfidf_similar_doc': dataset.iloc[tfidf_similarity_matrix[i, :].argmax()],
        'tfidf_similarity': tfidf_similarity_matrix[i, :].max(),
        'count_similar_doc': dataset.iloc[count_similarity_matrix[i, :].argmax()],
        'count_similarity': count_similarity_matrix[i, :].max(),
    }
    for i in [random.randint(0, len(dataset)) for _ in range(10)]
]    

In [29]:
%%html
<style>
    code {
        padding: 2px;
        color: blue;
    }
    .w-half {
        width: 50%;
    }
    .left {
        float: left;
    }
    .right {
        float: right;
    }
    .col {
        height: 100%;
        padding-left: 16px;
        padding-right: 16px;
        padding-top: 8px;
        padding-bottom: 8px;

    }
    .better {
        color: white;
        background-color: #5cb85c;
    }
    .worse {
        color: black;
        background-color: ;
    }
</style>

In [31]:
for r in result:
    title = r['doc'].title if r['doc'].title != '' else r['doc'].description[:140] + '...'
    movie = '<span>%(title)s</span> <span>%(description)s</span>'
    result_class = lambda vectorizer, r: 'better' if r[vectorizer + '_similarity'] > r[('tfidf' if vectorizer == 'count' else 'count') + '_similarity'] else 'worse' 
    desc_length = 280
    
    tfidf_content = movie % ({
        'title': r['tfidf_similar_doc'].title,
        'description': r['tfidf_similar_doc'].description[:desc_length] + '...',
    })
    count_content = movie % ({ 
        'title': r['count_similar_doc'].title,
        'description': r['count_similar_doc'].description[:desc_length] + '...',
    })
    
    display(HTML('''
    <div style="margin: 8px; border: 1px solid lightgray">
        <div style="text-align: center;background-color: whitesmoke;padding: 8px"><p r['doc'].title>''' + title + '''</p></div></div>
        <div>
            <div class="left w-half">
                <div class="col ''' + result_class('tfidf', r) + '''">
                    <div><strong>TF-IDF Vectorizer: </strong>''' + str(r['tfidf_similarity'])[:6] + '''</div>
                    <div>''' + tfidf_content + '''</div>
                </div>
            </div>
            <div class="right w-half">
                <div class="col ''' + result_class('count', r) + '''">
                <div><strong>Count Vectorizer: </strong>''' + str(r['count_similarity'])[:6] + '''</div>
                <div>''' + count_content + '''</div>
                </div>
            </div>
        </div>
    </div>
    '''))

###### Questão 2
Elabore um problema de classificação binária de textos coerente com sua
base.

a) Determine o rótulo dos documentos (separando os documentos em classes
bem definidas).

b) Extraia as representações vetoriais com CountVectorizer e TF-IDF.

c) Treine um classificador baseado em cada uma das duas representações vetoriais e Regressão Logística usando validação cruzada com 70% das amostras selecionadas para treino e 30% para teste. Exiba as matrizes de confusão,
métricas de acurácia, precisão, recall e F1 score.

d) Faça o mesmo para o classificador Naive-Bayes.

e) Faça o mesmo para um outro classificador de sua preferência (pesquise na
biblioteca Scikit-learn).

f) Compare os 6 resultados.


**Referências**

